In [ ]:
import os
import torch
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.models import ResNet50_Weights
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import confusion_matrix


In [ ]:
class WheatDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['healthy', 'sick']
        self.image_paths = []
        self.labels = []

        if not os.path.exists(root_dir):
            raise ValueError(f"Directory {root_dir} does not exist!")
        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.exists(class_dir):
                print(f"Warning: {class_dir} not found, skipping...")
                continue
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.jpg', '.png', '.jpeg', '.jfif')):
                    self.image_paths.append(os.path.join(class_dir, img_name))
                    self.labels.append(label)

        if len(self.image_paths) == 0:
            raise ValueError(f"No valid images found in {root_dir}!")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')  # Исправлено Image -> image

            if self.transform:
                image = self.transform(image)  # Image -> image
            
            return image, label
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None


In [ ]:
dataset = '/kaggle/input/wheatdataset/dataset/HealthySickWheat'
train_dir = '/kaggle/input/wheatdataset/dataset/HealthySickWheat/train'
test_dir = '/kaggle/input/wheatdataset/dataset/HealthySickWheat/test'

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.RandomHorizontalFlip(),  
    transforms.RandomRotation(10),     
    transforms.ToTensor(),              
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = WheatDataset(root_dir=train_dir, transform = train_transforms)
test_dataset = WheatDataset(root_dir=train_dir, transform = test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32 ,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32 ,shuffle=False)

In [ ]:
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def Train(model, dataloader, criterion, optimizer, epochs, device):
    model.train()

    train_losses = []
    train_accuracies = []
    
    for epoch in range(epochs):
        running_loss = 0
        correct = 0 
        total = 0
        for inputs, labels in tqdm(dataloader, desc='Training', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
        epoch_loss = running_loss / total
        epoch_acc = 100. * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        
        print(f"Epoch: {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuarcy: {epoch_acc:.2f}")

        plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Train Loss", color="blue")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label="Train Accuracy", color="green")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.title("Training Accuracy")
    plt.legend()

    plt.show()

In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc='Evaluating', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Сохраняем предсказания и истинные метки для матрицы ошибок
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    print(f'Evaluate: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.2f}%')

    # Построение матрицы ошибок
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Healthy', 'Sick'], yticklabels=['Healthy', 'Sick'])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()


In [ ]:
Train(model, train_loader, criterion , optimizer, 50,device)

In [ ]:
evaluate(model, test_loader, criterion, device)

In [ ]:
torch.save(model.state_dict(), "model_.pth")
print("Модель сохранена!")
